In [105]:
#print(__doc__)
from sklearn.linear_model import LogisticRegression
# Code source: Gaël Varoquaux
# Modified for documentation by Jaques Grobler
# License: BSD 3 clause
'''
import numpy as np
import matplotlib.pyplot as plt

from sklearn import datasets

# import some data to play with
iris = datasets.load_iris()
X = iris.data[:, :2]  # we only take the first two features.
print(X)
Y = iris.target
print(Y)

logreg = LogisticRegression(C=1e5, solver='lbfgs', multi_class='multinomial')

# Create an instance of Logistic Regression Classifier and fit the data.
logreg.fit(X, Y)

# Plot the decision boundary. For that, we will assign a color to each
# point in the mesh [x_min, x_max]x[y_min, y_max].
x_min, x_max = X[:, 0].min() - .5, X[:, 0].max() + .5
y_min, y_max = X[:, 1].min() - .5, X[:, 1].max() + .5
h = .02  # step size in the mesh
xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
Z = logreg.predict(np.c_[xx.ravel(), yy.ravel()])

# Put the result into a color plot
Z = Z.reshape(xx.shape)
plt.figure(1, figsize=(4, 3))
plt.pcolormesh(xx, yy, Z, cmap=plt.cm.Paired)

# Plot also the training points
plt.scatter(X[:, 0], X[:, 1], c=Y, edgecolors='k', cmap=plt.cm.Paired)
plt.xlabel('Sepal length')
plt.ylabel('Sepal width')

plt.xlim(xx.min(), xx.max())
plt.ylim(yy.min(), yy.max())
plt.xticks(())
plt.yticks(())

plt.show()
'''


"\nimport numpy as np\nimport matplotlib.pyplot as plt\n\nfrom sklearn import datasets\n\n# import some data to play with\niris = datasets.load_iris()\nX = iris.data[:, :2]  # we only take the first two features.\nprint(X)\nY = iris.target\nprint(Y)\n\nlogreg = LogisticRegression(C=1e5, solver='lbfgs', multi_class='multinomial')\n\n# Create an instance of Logistic Regression Classifier and fit the data.\nlogreg.fit(X, Y)\n\n# Plot the decision boundary. For that, we will assign a color to each\n# point in the mesh [x_min, x_max]x[y_min, y_max].\nx_min, x_max = X[:, 0].min() - .5, X[:, 0].max() + .5\ny_min, y_max = X[:, 1].min() - .5, X[:, 1].max() + .5\nh = .02  # step size in the mesh\nxx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))\nZ = logreg.predict(np.c_[xx.ravel(), yy.ravel()])\n\n# Put the result into a color plot\nZ = Z.reshape(xx.shape)\nplt.figure(1, figsize=(4, 3))\nplt.pcolormesh(xx, yy, Z, cmap=plt.cm.Paired)\n\n# Plot also the training points\

In [94]:
UP = []
DOWN = []

def count_words(original_string, bag_of_words):
    split_string = original_string.lower().split()
    count = 0
    for word in bag_of_words:
        matches = split_string.count(word)
        count += matches
    ratio = count/len(split_string)
    return ratio
    
def aggregate_jsons(json_list):
    result = []
    for json in json_list:
        headline = json["title"]
        desc = json["description"]
        x = headline + " " + desc
        result.append(x)
    return result

def pipeline(df_path, ticky):
    # get the dataframe from csv
    df = pd.read_csv(df_path)
    
    # fix dates
    df["Date"] = df["Date"].apply(lambda x: pd.to_datetime(x, infer_datetime_format=True))
    df["Start"] = df["Date"].apply(str)
    six_days = lambda start_date: start_date + datetime.timedelta(days=6)
    df["End"] = df["Date"].apply(six_days).apply(str)
    df = df.drop(columns=["Date"])
    remove_time = lambda dt: dt[0:10]
    df["Start"] = df["Start"].apply(remove_time)
    df["End"] = df["End"].apply(remove_time)
    
    # add json
    client = TiingoClient({"api_key": "a265fc4a1013923f970d16e7348195074e97fcb0"})
    query_ticker = lambda t, s, e: client.get_news(tickers=[t], startDate=s, endDate=e)
    df["JSON"] = df.apply(lambda d: query_ticker(ticky, d["Start"], d["End"]), axis=1)
    
    # create corpus
    df["corpus"] = df["JSON"].apply(aggregate_jsons)
    df["vectorized"] = df["corpus"]
    combinatric = lambda l: ''.join(l)
    df["corpus"] = df["vectorized"].apply(combinatric)
    indicoio.config.api_key = "25b83c4c388204edd2c6c11cd907e048"
    # add sentiment
    df["sentiment"] = df["corpus"].apply(lambda orig: TextBlob(orig).sentiment.polarity)
    df["sentiment_test"] = df["vectorized"].apply(lambda orig: [TextBlob(o).sentiment.polarity for o in orig]).apply(np.mean)
    #df["indico_sentiment"] = df["corpus"].apply(lambda text: indicoio.sentiment_hq(text))
    # add statistical features
    # df["features"] = df["corpus"].apply(lambda text: indicoio.text_features(text, v=2))
    
    # add bag of words ratio
    #df["bad_ratio"] = 
    #df["good_keywords"]
    
    # return the nice beautiful dataframe
    return df

In [110]:
def predictor(d, ticky):
    from sklearn.model_selection import train_test_split
    from sklearn import linear_model

    d["constvec"] = [0.7] * len(d)
    d["lastweek"] = d["Close"]
    df.lastweek = df.lastweek.shift(7) ## shift down
    #df.lastweek.drop(df.gdp.shape[0] - 1,inplace = True) ## removing the first row
    X = d[["Open", "lastweek", "indico_sentiment"]]
    Y = d["Close"]
    imp = SimpleImputer(missing_values=np.nan, strategy='mean')
    imp = imp.fit(X)
    X = imp.transform(X)
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.3, random_state = 69)

    def mse(v1, v2):
        return np.sum((v1 - v2) ** 2) 
    
    model = linear_model.BayesianRidge()
    model.fit(X_train, Y_train)
    Y_predicted = model.predict(X_test)
    error = mse(Y_predicted, Y_test)
    print("aggregated error : {0}".format(error))
    print("average error by prediction for {1}: {0}".format(np.mean(Y_predicted - Y_test), ticky))
    
    
    
def predictor_two(d, ticky):
    import matplotlib.pyplot as plot
    from sklearn import preprocessing
    from sklearn.naive_bayes import GaussianNB
    from sklearn import utils
    d["lastweek"] = d["Close"]
    d.lastweek = d.lastweek.shift(7) ## shift down
    #df.lastweek.drop(df.gdp.shape[0] - 1,inplace = True) ## removing the first row
    #X = np.c_[np.array(d["Open"]), np.array(d["indico_sentiment"])]
    print(d.head())
    X = np.vstack((d["Open"],  d["sentiment"])).T
    Y = d["Adj Close"]
    Y = (Y - d["lastweek"]) > 0
    # import some data to play with
    print(X)
    print("X length:")
    print(X.size)
    print(Y)
    print("Y length:")
    print(Y.size)
    logreg = LogisticRegression(C=1e5, solver='lbfgs', multi_class='multinomial')

    # Create an instance of Logistic Regression Classifier and fit the data.
    logreg.fit(X, Y)

    # Plot the decision boundary. For that, we will assign a color to each
    # point in the mesh [x_min, x_max]x[y_min, y_max].
    x_min, x_max = X[:, 0].min() - .5, X[:, 0].max() + .5
    y_min, y_max = X[:, 1].min() - .5, X[:, 1].max() + .5
    h = .02  # step size in the mesh
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
    Z = logreg.predict(np.c_[xx.ravel(), yy.ravel()])

    # Put the result into a color plot
    Z = Z.reshape(xx.shape)
    plt.figure(1, figsize=(4, 3))
    plt.pcolormesh(xx, yy, Z, cmap=plt.cm.Paired)

    # Plot also the training points
    plt.scatter(X[:, 0], X[:, 1], c=Y, edgecolors='k', cmap=plt.cm.Paired)
    plt.xlabel('OPEN')
    plt.ylabel('SENTIMENT')

    plt.xlim(xx.min(), xx.max())
    plt.ylim(yy.min(), yy.max())
    plt.xticks(())
    plt.yticks(())

    plt.show()



    

In [37]:
start = time.time()
df = pipeline('aapl-6m-weekly.csv', 'aapl')
end = time.time()
pipeline_time = end - start
print("pipeline took {0}s for {1}".format(pipeline_time, 'aapl'))
predictor(df, 'aapl')

pipeline took 31.01529049873352s for aapl
aggregated error : 158.50752155650392
average error by prediction for aapl: 0.08794019001604486


In [ ]:
df = pipeline('aapl-6m-weekly.csv', 'aapl')
predictor_two(df, 'aapl')